# PEFT configurations and models

Genellikle milyarlarca parametreye sahip olan günümüzün büyük ön eğitimli modellerinin büyük boyutu, tüm bu hesaplamaları yapmak için daha fazla depolama alanı ve daha fazla hesaplama gücü gerektirdiğinden önemli bir eğitim zorluğu oluşturmaktadır. Bu büyük ön eğitimli modelleri eğitmek için güçlü GPU'lara veya TPU'lara erişmeniz gerekir ki bu da pahalı, herkes için yaygın olarak erişilebilir değil, çevre dostu değil ve çok pratik değil. PEFT yöntemleri bu zorlukların çoğunu ele alır. Çeşitli PEFT yöntemleri (yumuşak yönlendirme, matris ayrıştırma, adaptörler) vardır, ancak hepsi aynı şeye odaklanır, eğitilebilir parametrelerin sayısını azaltır. Bu, büyük modellerin tüketici donanımında eğitilmesini ve depolanmasını daha erişilebilir hale getirir.

PEFT kütüphanesi, büyük modelleri ücretsiz veya düşük maliyetli GPU'larda hızlı bir şekilde eğitmenize yardımcı olmak için tasarlanmıştır ve bu eğitimde, eğitim için önceden eğitilmiş bir temel modele bir PEFT yöntemi uygulamak için bir yapılandırmanın nasıl kurulacağını öğreneceksiniz. PEFT yapılandırması kurulduktan sonra, istediğiniz herhangi bir eğitim çerçevesini kullanabilirsiniz (Transformer'ın Trainer sınıfı, Accelerate, özel bir PyTorch eğitim döngüsü).

## PEFT configurations

Bir yapılandırma, belirli bir PEFT yönteminin nasıl uygulanması gerektiğini belirten önemli parametreleri depolar.

Örneğin, LoRA uygulamak için aşağıdaki **LoraConfig** ve **p-tuning** uygulamak için **PromptEncoderConfig** dosyalarına bir göz atın (bu yapılandırma dosyaları zaten JSON-serileştirilmiştir). Bir PEFT adaptörü yüklediğinizde, gerekli olan ilişkili bir **adapter_config.json** dosyasına sahip olup olmadığını kontrol etmek iyi bir fikirdir.

LoraConfig
```
{
  "base_model_name_or_path": "facebook/opt-350m", #base model to apply LoRA to
  "bias": "none",
  "fan_in_fan_out": false,
  "inference_mode": true,
  "init_lora_weights": true,
  "layers_pattern": null,
  "layers_to_transform": null,
  "lora_alpha": 32,
  "lora_dropout": 0.05,
  "modules_to_save": null,
  "peft_type": "LORA", #PEFT method type
  "r": 16,
  "revision": null,
  "target_modules": [
    "q_proj", #model modules to apply LoRA to (query and value projection layers)
    "v_proj"
  ],
  "task_type": "CAUSAL_LM" #type of task to train model on
}
```

PromptEncoderConfig
```
{
  "base_model_name_or_path": "roberta-large", #base model to apply p-tuning to
  "encoder_dropout": 0.0,
  "encoder_hidden_size": 128,
  "encoder_num_layers": 2,
  "encoder_reparameterization_type": "MLP",
  "inference_mode": true,
  "num_attention_heads": 16,
  "num_layers": 24,
  "num_transformer_submodules": 1,
  "num_virtual_tokens": 20,
  "peft_type": "P_TUNING", #PEFT method type
  "task_type": "SEQ_CLS", #type of task to train model on
  "token_dim": 1024
}
```

In [2]:
!pip install -q peft

In [3]:
from peft import PromptEncoderConfig, TaskType

p_tuning_config = PromptEncoderConfig(
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=128,
    num_attention_heads=16,
    num_layers=24,
    num_transformer_submodules=1,
    num_virtual_tokens=20,
    token_dim=1024,
    task_type=TaskType.SEQ_CLS
)

In [6]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    r=16,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=32,
    lora_dropout=0.05
)

## PEFT models

Elinizde bir PEFT yapılandırması varken, artık bunu bir **PeftModel** oluşturmak için önceden eğitilmiş herhangi bir modele uygulayabilirsiniz. Transformers kitaplığındaki son teknoloji modellerden, özel bir modelden ve hatta yeni ve desteklenmeyen transformatör mimarilerinden birini seçin.

Bu eğitimde, ince ayar yapmak için temel bir facebook/opt-350m modeli yükleyin.

In [4]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Temel facebook/opt-350m modelinden ve daha önce oluşturduğunuz **lora_config**'den bir **PeftModel** oluşturmak için **get_peft_model()** işlevini kullanın.

In [7]:
from peft import get_peft_model

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 332,769,280 || trainable%: 0.4727


Artık PeftModel'i tercih ettiğiniz eğitim çerçevesi ile eğitebilirsiniz! Eğitimden sonra, modelinizi save_pretrained() ile yerel olarak kaydedebilir veya push_to_hub yöntemiyle Hub'a yükleyebilirsiniz.

In [9]:
# save locally
lora_model.save_pretrained("Leotrim/opt-350m-lora")

# push to Hub
lora_model.push_to_hub("Leotrim/opt-350m-lora")

Çıkarım için bir PeftModel yüklemek için, onu oluşturmak için kullanılan PeftConfig'i ve eğitildiği temel modeli sağlamanız gerekir.

In [ ]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained("ybelkada/opt-350m-lora")
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
lora_model = PeftModel.from_pretrained(model, "ybelkada/opt-350m-lora")

>Varsayılan olarak, PeftModel çıkarım için ayarlanmıştır, ancak bağdaştırıcıyı biraz daha eğitmek isterseniz **is_trainable=True** ayarını yapabilirsiniz.

`lora_model = PeftModel.from_pretrained(model, "ybelkada/opt-350m-lora", is_trainable=True)`

PeftModel.from_pretrained() yöntemi bir PeftModel yüklemenin en esnek yoludur çünkü hangi model çerçevesinin kullanıldığı önemli değildir (Transformers, timm, genel bir PyTorch modeli). AutoPeftModel gibi diğer sınıflar, temel PeftModel etrafında kullanışlı bir sarmalayıcıdır ve PEFT modellerini doğrudan Hub'dan veya yerel olarak PEFT ağırlıklarının depolandığı yerden yüklemeyi kolaylaştırır.

In [10]:
from peft import AutoPeftModelForCausalLM

lora_model = AutoPeftModelForCausalLM.from_pretrained("ybelkada/opt-350m-lora")

adapter_config.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]